In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt


import torch
from startorch import example as ex
from startorch import transformer as tr
from startorch import timeseries as ts
from startorch import sequence as seq
from startorch import tensor as te
from startorch.utils.seed import get_torch_generator

In [3]:
plt.style.use("bmh")
plt.rcParams["figure.figsize"] = (16, 5)

In [4]:
random_seed = 15803713921897248048

# Table of content

## Markov + Exponential
[top](#Table-of-content)

In [5]:
rng = get_torch_generator(random_seed)
num_classes = 10
actions = torch.arange(num_classes)

transition = torch.diag(torch.ones(num_classes))
transition = transition[torch.randperm(num_classes)]
# transition = mask_by_row(torch.rand(num_classes, num_classes), n=num_classes - 4, rng=rng)
print(transition)

weights = torch.logspace(start=0.1, end=1.0, steps=num_classes)

generator = ex.Transform(
    generator=ex.TimeSeriesExampleGenerator(
        generators=ts.SequenceTimeSeriesGenerator(
            # generators={"action": seq.RandInt(0, num_classes)},
            generators={"action": seq.MarkovChain(transition=transition)},
        ),
        seq_len=te.Full(128),
    ),
    transformer=tr.Sequential(
        [
            tr.LookupTable(weights=weights, index="action", output="rate"),
            tr.Exponential(rate="rate", output="time")
        ]
    ),
)
print(generator)

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])
TransformExampleGenerator(
  (generator): TimeSeriesExampleGenerator(
      (generators): SequenceTimeSeriesGenerator(
          (action): MarkovChainSequenceGenerator(transition=(10, 10), init=None)
        )
      (seq_len): FullTensorGenerator(value=128, dtype=None)
    )
  (transformer): SequentialTransformer(
      (0): LookupTableTransformer(size=10, index=action, output=rate, exist_ok=False)
      (1): ExponentialTransformer(rate=rate, output=time, exist_ok=False)
    )
)


In [6]:
batch = generator.generate(batch_size=10000, rng=rng)

for key, value in batch.items():
    print(key, tuple(value.shape))

action = batch["action"].numpy().ravel()
time = batch["time"].numpy().ravel()

action (10000, 128)
rate (10000, 128)
time (10000, 128)


In [ ]:
# Analyze the action distribution
action_transition = torch.zeros(num_classes, num_classes)
for i, j in zip(action[:-1], action[1:], strict=True):
    action_transition[i, j] += 1
action_transition = action_transition / action_transition.sum()
fig, axes = plt.subplots(figsize=(16, 8), ncols=2)
_ = axes[0].matshow(transition)
_ = axes[1].matshow(action_transition)

In [ ]:
# Analyze the time distribution
fig, ax = plt.subplots(figsize=(16, 8))
_ = ax.scatter(time, action, alpha=0.1, color="tab:cyan", label="sample")
_ = ax.scatter([time[action == i].mean().item() for i in range(num_classes)], actions, color="tab:red", label="mean (estimated)", marker="x")
_ = ax.scatter((1 / weights).numpy(), actions.numpy(), color="black", label="mean (exact)", marker="x")
_ = ax.legend()